In [1]:

import lightgbm as lgb
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用

import data_get
import race_before_data
import pickle
import time
import before_odds

from sklearn.metrics import recall_score

In [17]:
date = '2022-04-05'
data_get.download_file('racelists', date)
df_race = data_get.get_racelists(date)

df_race.to_csv(f"dataset/rawdata_syussouhyou/{date}_race_list.csv", encoding='utf_8_sig')
df_race = df_race.sort_values(by="deadline")

data_get.download_file('results', date)
df_result = data_get.get_results(date)
kane = 0
tousi = 0

odds_in_kane = 0

filename = '0402model.sav'
model = pickle.load(open(filename, 'rb'))

def hensachi(x):
    x = x.astype(float)
    if x.std() == 0:
        return pd.Series(np.full(6,50.0))

    hensa = (x - x.mean())*10/x.std()+50

    return hensa

for  index, row in df_race.iterrows():


    try:
        time.sleep(1)
        place_cd , race_no, date = row["place_cd"] ,row["race_no"], row["date"]
        bi = race_before_data.get_beforeinfo(date, place_cd, race_no)
        tmp = pd.merge(pd.DataFrame(row).T, pd.DataFrame(bi).T, on=["race_no", "place_cd"])
        usecols = [f'{k}_{i}' for k in ('class', 'glob_win', 'glob_in2',
                                    'loc_win', 'loc_in2', 'moter_in2', 'boat_in2', 'EST','ESC' )
                for i in range(1, 7)]
        usecols += [f'ET_{i}'
                    for i in range(1, 7)]

        tmp = tmp[usecols]

        def rank_mapper(x):
            if 'A1' in x:
                y = 4
            elif 'A2' in x:
                y = 3
            elif 'B1' in x:
                y = 2
            else:
                y = 1
            return y

        for i in range(1,7):
            col_name_class = f"class_{i}"
            tmp[col_name_class] = tmp[col_name_class].map(rank_mapper)
        class_col = [s for s in tmp.columns if "class" in s]


        tmp.loc[:, class_col] = tmp.loc[:,class_col].apply(hensachi,axis=1)

        hensa_col = ['glob_win', 'glob_in2','loc_win', 'loc_in2', 'moter_in2', 'boat_in2', 'EST', "ET","ESC"]

        for i in hensa_col:
            class_col = [s for s in tmp.columns if i in s]
            tmp[class_col] = tmp.loc[:, class_col].apply(hensachi,axis=1)


        X = tmp.values
        y_pred = model.predict(X)
        # テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率...] を返す)
        y_pred_prob = model.predict_proba(X)

        result_row = df_result[(df_result['place_cd'] == place_cd) & (df_result['race_no'] == race_no)]
        odds = result_row["odds_1t"].values[0]
        resu = result_row["tkt_1t"].values[0]
        hoge = before_odds.get_odds(BET_TYPE="tf",race_no=race_no,place_cd=place_cd, date=date)

        before_odds_list = []

        for i in hoge.select("td.oddsPoint"):
            before_odds_list.append(i.text)

        hoge = [float(s) for s in before_odds_list]



        # print(f"予測確率：{y_pred_prob}")
        # print(f"直前オッズ:{hoge}")
        # print(f"オッズ＊予測{hoge*y_pred_prob}")
        oddsinpred = np.argmax(hoge * y_pred_prob)
        print(f"オッズ込み予想{oddsinpred +1}, 結果：{resu} ,オッズ{hoge[oddsinpred]}")
        # print(f"予想：{y_pred},結果：{resu},オッズ：{odds}")

        # # 勝率のみ回収率
        # kane -= 100
        # if y_pred[0] == int(resu):
        #     kane += odds * 100
        # print(kane)

        # オッズ込み回収率
        print(np.max(hoge * y_pred_prob))
        if resu == "特払い":
            print("特払い")

        elif np.max(hoge * y_pred_prob) > 1.5:
            odds_in_kane -= 100
            tousi += 100
            if (int(oddsinpred)+1) == int(resu):
                print(odds)
                print(f"当選金：{odds * 100}")
                odds_in_kane += odds * 100
                kane += odds * 100
        print(odds_in_kane)

    except Exception as e:
        print(e, flush=True)


print(f"使った金額{tousi}")
print(f"当選金額{kane}")



オッズ込み予想4, 結果：1 ,オッズ32.2
2.5122728143366224
-100
オッズ込み予想3, 結果：1 ,オッズ13.1
0.8842927614618225
-100
オッズ込み予想5, 結果：1 ,オッズ12.7
1.6332529026093785
-200
オッズ込み予想3, 結果：2 ,オッズ13.3
1.3372423908719808
-200
オッズ込み予想1, 結果：1 ,オッズ1.1
0.9278403291640869
-200
オッズ込み予想3, 結果：1 ,オッズ7.9
2.893423177799882
-300
オッズ込み予想2, 結果：1 ,オッズ16.8
1.6980665486656945
-400
オッズ込み予想3, 結果：6 ,オッズ9.5
1.4512479217369656
-400
オッズ込み予想1, 結果：1 ,オッズ1.9
1.0114605366591791
-400
オッズ込み予想3, 結果：2 ,オッズ6.6
0.9532355757012171
-400
オッズ込み予想2, 結果：2 ,オッズ3.0
1.4534759133889699
-400
オッズ込み予想3, 結果：1 ,オッズ45.1
4.025093482304471
-500
オッズ込み予想5, 結果：1 ,オッズ25.9
2.7109620722253718
-600
オッズ込み予想1, 結果：1 ,オッズ2.7
1.1752671242862214
-600
オッズ込み予想3, 結果：2 ,オッズ12.0
1.5399505802423594
-700
オッズ込み予想2, 結果：6 ,オッズ24.4
6.014195850058474
-800
オッズ込み予想4, 結果：3 ,オッズ8.0
1.8417656912115943
-900
オッズ込み予想1, 結果：1 ,オッズ3.5
1.3673341848851297
-900
オッズ込み予想3, 結果：2 ,オッズ9.0
1.2011429789046217
-900
オッズ込み予想2, 結果：3 ,オッズ22.7
1.536734782431087
-1000
オッズ込み予想2, 結果：2 ,オッズ8.7
1.8203124102209418
8.7
当選金：869

In [29]:
result_row["odds_1t"].values[0]

13.2

In [38]:

df_race["deadline"]

0      08:59
1      09:28
2      09:57
3      10:26
43     10:30
       ...  
113    20:05
41     20:14
114    20:37
23     20:39
42     20:43
Name: deadline, Length: 115, dtype: object

In [39]:
df_race

,title,day,date,place_cd,place,race_no,race_type,distance,deadline,toban_1,...,boat_no_3,boat_no_4,boat_no_5,boat_no_6,boat_in2_1,boat_in2_2,boat_in2_3,boat_in2_4,boat_in2_5,boat_in2_6
0,日本MB選手会会長杯,3,2022-04-03,21,芦屋,1,予選,1800,08:59,3606,...,33,57,40,45,34.62,58.10,31.51,37.44,33.78,35.74
1,日本MB選手会会長杯,3,2022-04-03,21,芦屋,2,予選,1800,09:28,3582,...,56,14,23,20,34.56,32.87,30.59,31.78,33.33,32.86
2,日本MB選手会会長杯,3,2022-04-03,21,芦屋,3,予選,1800,09:57,4384,...,19,67,58,73,36.00,39.57,33.04,31.66,30.73,30.84
3,日本MB選手会会長杯,3,2022-04-03,21,芦屋,4,予選,1800,10:26,4189,...,26,31,11,21,31.42,34.91,34.08,31.94,35.42,37.44
43,BTS養老開設5周年記念よーたんカップ,6,2022-04-03,9,津,1,ツッキーレー,1800,10:30,4191,...,13,61,15,38,33.09,26.89,35.77,39.16,37.23,27.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,ルーキーシリーズ第7戦第15回ドラキリュウカップ,2,2022-04-03,1,桐生,11,予選特選,1800,20:05,5043,...,49,15,56,27,22.08,55.13,44.44,26.32,27.54,36.84
41,Bカードメンバー大感謝祭日本トーター杯,7,2022-04-03,15,丸亀,11,特別選抜戦,1800,20:14,3556,...,8,36,40,64,33.78,32.64,37.75,43.97,24.83,35.81
114,ルーキーシリーズ第7戦第15回ドラキリュウカップ,2,2022-04-03,1,桐生,12,桐生特選,1800,20:37,4769,...,21,20,12,47,44.44,45.00,39.19,30.26,27.50,35.44
23,シャボン玉石けん杯,3,2022-04-03,20,若松,12,予選特選,1800,20:39,3388,...,47,70,36,29,26.82,36.31,32.45,32.00,28.32,34.12


In [41]:

hoge

[1.9, 12.5, 4.0, 3.2, 7.3, 16.1]

In [3]:
df_result = data_get.get_results(date)


In [9]:

df_result